In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import re

In [2]:
os.getcwd()

'C:\\Users\\shilp\\Desktop\\PhDThesis\\Congress\\Networkgraphs'

In [3]:
#os.chdir("C:\\Users\\ankur\\Desktop\\shilpi")

In [4]:
df_Dem = pd.read_csv('Clean_7thNov_6thDec2017_RepDemOfficial.csv')

In [5]:
#df_Rep = pd.read_csv('Clean_7thNov_6thDec2017_RepRepOfficial.csv')

In [6]:
#df = df_Dem.append(df_Rep)
df = df_Dem

In [7]:
df = df.sort_values("AUTHOR")

In [8]:
def find_mentioned(tweet):
    '''This function will extract the twitter handles of people mentioned in the tweet'''
    return re.findall('(?<!RT\s)(@[A-Za-z]+[A-Za-z0-9-_]+)', tweet)  

In [9]:
df['mentioned'] = df.CONTENT.apply(find_mentioned)

In [10]:
df2 = df[['AUTHOR', 'mentioned']]

In [11]:
df2 = df2.groupby(['AUTHOR'],as_index=False).agg({'mentioned': 'sum'})

In [12]:
df2['mentioned'] = df2['mentioned'].map(lambda x: "".join(x))

In [13]:
df2['mentioned'] = df2['mentioned'].map(lambda x: x.lstrip('@').replace('@', ','))

In [14]:
#split row into multiple rows based on same value of a column
import numpy as np
from itertools import chain
# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))
# calculate lengths of splits
lens = df2['mentioned'].str.split(',').map(len)
# create new dataframe, repeating or chaining as appropriate
res = pd.DataFrame({'AUTHOR': np.repeat(df2['AUTHOR'], lens),'mentioned': chainer(df2['mentioned'])})

In [15]:
res = res.apply(lambda x: x.astype(str).str.upper())

In [16]:
res = res[res.AUTHOR != res.mentioned]

In [17]:
res = res.rename(columns={"AUTHOR": "Source", "mentioned": "Target"})

In [18]:
#df3 = df2['mentioned'].str.split(',',expand=True)

In [19]:
#df3.insert(0, "AUTHOR", df2['AUTHOR'])

In [20]:
#df3.to_csv (r'7thNov_6thDec2017_Network.csv', index = None, header=False)

In [21]:
res.to_csv (r'7thNov_6thDec2017_Network_Dem.csv', index = None, header=True)

In [22]:
df_label =  pd.DataFrame(res.Source.unique(), columns = ['Source'])

In [23]:
df_label =  df_label.append(pd.DataFrame(res.Target.unique(), columns = ['Source']))

In [24]:
df_label['Label'] = 'U'

In [25]:
#demdf = pd.DataFrame(df_Dem.AUTHOR.unique(), columns = ['Source'])
#repdf = pd.DataFrame(df_Rep.AUTHOR.unique(), columns = ['Source'])
demdf = pd.DataFrame(df.AUTHOR.unique(), columns = ['Source'])

In [26]:
#df_label.loc[df_label.Source.isin(demdf.Source), ['Label']] = 'D'
df_label.loc[df_label.Source.isin(demdf.Source), ['Label']] = 'D'

In [27]:
#df_label.loc[df_label.Source.isin(repdf.Source), ['Label']] = 'R'

In [28]:
df_label.to_csv (r'7thNov_6thDec2017_Attribute_Dem.csv', index = None, header=True)